# AMZN Stock Article extractions

In [1]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = 'Amazon OR "AMZN" OR "Jeff Bezos" OR "Andy Jassy" OR "E-commerce" OR "Amazon Web Services" OR "AWS" OR "Amazon Prime" OR "Amazon marketplace" OR "Whole Foods" OR "Amazon logistics"'

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2023-12-18, Headline: Jeff Bezos describes his perfect meeting
Date: 2023-12-27, Headline: Deathloop is free on PC for Amazon Prime members
Date: 2023-12-18, Headline: Jeff Bezos wants Elon Musk to know Blue Origin is serious now
Date: 2023-12-11, Headline: Is Amazon's Jeff Bezos a bad neighbor?
Date: 2023-12-19, Headline: 7 Amazon retail stumbles, from grocery delivery to clothing stores
Date: 2023-12-15, Headline: Jeff Bezos says compromising with coworkers is actually a bad idea, because it 'doesn't lead to truth'
Date: 2023-12-28, Headline: 10 great shows from 2023 to stream on Amazon Prime Video
Date: 2023-12-29, Headline: The top 7 people in artificial-intelligence e-commerce
Date: 2023-12-11, Headline: Walmart is going after Amazon in a new way — and borrowing a strategy from Target
Date: 2023-12-21, Headline: A new group at Amazon is helping customers including PGA Tour and Bridgewater make generative-AI applications
Date: 2023-12-15, Headline: Jeff Bezos says the main re

# Sentiment Analysis on News Articles

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
sentiment_data

[{'date': '2023-12-18',
  'headline': 'Jeff Bezos describes his perfect meeting',
  'sentiment_positive': 0.9999908208847046,
  'sentiment_neutral': 2.2704950879415264e-06,
  'sentiment_negative': 6.964671229070518e-06},
 {'date': '2023-12-27',
  'headline': 'Deathloop is free on PC for Amazon Prime members',
  'sentiment_positive': 0.9999005794525146,
  'sentiment_neutral': 5.421298192231916e-05,
  'sentiment_negative': 4.5225249778013676e-05},
 {'date': '2023-12-18',
  'headline': 'Jeff\xa0Bezos\xa0wants\xa0Elon Musk\xa0to know Blue Origin is serious now',
  'sentiment_positive': 0.9647932052612305,
  'sentiment_neutral': 0.00013012185809202492,
  'sentiment_negative': 0.03507668524980545},
 {'date': '2023-12-11',
  'headline': "Is Amazon's Jeff Bezos a bad neighbor?",
  'sentiment_positive': 0.9921162724494934,
  'sentiment_neutral': 0.00020694363047368824,
  'sentiment_negative': 0.007676729932427406},
 {'date': '2023-12-19',
  'headline': '7 Amazon retail stumbles, from grocery de

# Historical Stock Data and Sentiment Analysis Combined

In [5]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'AMZN'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11 -1.446727 -1.794926 -1.623137 -1.414004  -1.414004  0.169651   
2023-12-12 -1.489308 -1.375417 -1.129152 -0.915007  -0.915007 -0.154334   
2023-12-13 -0.698527 -0.747751 -0.676828 -0.488190  -0.488190  0.270650   
2023-12-14 -0.148020 -0.401900 -1.063684 -0.933836  -0.933836  0.576793   
2023-12-15 -0.619445 -0.392288 -0.361394 -0.133554  -0.133554  3.382435   
2023-12-18  0.043594  0.978324  0.284355  1.153174   1.153174  0.800208   
2023-12-19  1.211520  1.064784  1.069967  1.065296   1.065296 -0.250666   
2023-12-20  0.755298  1.228108  0.733700  0.541191   0.541191  0.137856   
2023-12-21  0.876960  0.696514  0.894396  1.080989   1.080989 -0.623691   
2023-12-22  1.019910  0.818206  1.075920  0.949178   0.949178 -0.994544   
2023-12-26  0.956037  0.699715 

<ipython-input-5-260e35ffde9f>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


In [6]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,-1.446727,-1.794926,-1.623137,-1.414004,-1.414004,0.169651,0.958915,0.038476,0.002609
2023-12-12,-1.489308,-1.375417,-1.129152,-0.915007,-0.915007,-0.154334,0.798439,0.170789,0.030772
2023-12-13,-0.698527,-0.747751,-0.676828,-0.488190,-0.488190,0.270650,0.637964,0.303102,0.058935
2023-12-14,-0.148020,-0.401900,-1.063684,-0.933836,-0.933836,0.576793,0.987417,0.000031,0.012553
2023-12-15,-0.619445,-0.392288,-0.361394,-0.133554,-0.133554,3.382435,0.836596,0.032695,0.130709
2023-12-18,0.043594,0.978324,0.284355,1.153174,1.153174,0.800208,0.750716,0.000034,0.249250
2023-12-19,1.211520,1.064784,1.069967,1.065296,1.065296,-0.250666,0.588060,0.000027,0.411913
2023-12-20,0.755298,1.228108,0.733700,0.541191,0.541191,0.137856,0.652548,0.185543,0.161909
2023-12-21,0.876960,0.696514,0.894396,1.080989,1.080989,-0.623691,0.662005,0.333170,0.004825


# Transformer Architecture

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 326ms/step - loss: 1.0155 - val_loss: 0.0105
Epoch 2/50
2/2 [==============================] - 0s 39ms/step - loss: 0.2915 - val_loss: 0.1903
Epoch 3/50
2/2 [==============================] - 0s 38ms/step - loss: 0.3079 - val_loss: 0.1371
Epoch 4/50
2/2 [==============================] - 0s 42ms/step - loss: 0.3376 - val_loss: 0.0246
Epoch 5/50
2/2 [==============================] - 0s 44ms/step - loss: 0.2210 - val_loss: 0.0075
Epoch 6/50
2/2 [==============================] - 0s 42ms/step - loss: 0.1349 - val_loss: 0.0447
Epoch 7/50
2/2 [==============================] - 0s 42ms/step - loss: 0.0813 - val_loss: 0.0670
Epoch 8/50
2/2 [==============================] - 0s 66ms/step - loss: 0.1684 - val_loss: 0.0385
Epoch 9/50
2/2 [==============================] - 0s 45ms/step - loss: 0.1173 - val_loss: 0.0107
Epoch 10/50
2/2 [==============================] - 0s 44ms/step - loss: 0.0793 - val_loss: 0.0017
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 93ms/step
MSE: 0.055714133333468974, MAE: 0.20685555305933606, R^2: 0.8853601834402264


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
